In [1]:
import nltk
from nltk import word_tokenize
from nltk.lm.preprocessing import padded_everygram_pipeline, flatten, everygrams, pad_both_ends
from collections import Counter
from nltk.lm import Laplace, KneserNeyInterpolated, WittenBellInterpolated, StupidBackoff
import pandas as pd
import nltk
# nltk.download('punkt')
import pandas as pd
from nltk import FreqDist, KneserNeyProbDist
from nltk.util import bigrams, trigrams
from nltk.util import ngrams
from nltk.lm import NgramCounter
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk import word_tokenize, sent_tokenize
from nltk.lm import MLE
from nltk.lm.api import LanguageModel, Smoothing
from nltk.lm.smoothing import AbsoluteDiscounting, KneserNey, WittenBell
import numpy as np
from sklearn.model_selection import train_test_split
import nltk.lm as lm

In [2]:
FILE_PATH = 'data/hate/train_text.txt'

In [3]:
with open(FILE_PATH, 'r', encoding = 'utf-8') as f:
    dataset = f.readlines()
    
split = round(len(dataset)*0.8)
train_data = dataset[:split]
test_data = dataset[split:]

In [4]:
train_tokenized_data = [list(map(str.lower, nltk.tokenize.word_tokenize(sent))) 
                for sent in train_data]

test_tokenized_data = [list(map(str.lower, nltk.tokenize.word_tokenize(sent))) 
                for sent in test_data]

test_tokenized_data = [sent for sent in test_tokenized_data if sent]

In [5]:
n = 3

train, vocab = padded_everygram_pipeline(n, train_tokenized_data)
# test, vocab = padded_everygram_pipeline(n, test_tokenized_data)
test = list(ngrams(list(pad_both_ends(test_tokenized_data, n=n)), n))

kn = KneserNeyInterpolated(n) 
kn.fit(train, vocab)

In [6]:
for i, t in enumerate(test[:50]):
    try:
        val = kn.perplexity(t)
    except ZeroDivisionError:
        val = "tu było 0"
    except TypeError:
        val = "x"
    print(f"index: {i}, perplexity: {val}") if (val == np.inf or val == "x") else None

index: 6, perplexity: inf
index: 7, perplexity: inf
index: 8, perplexity: inf
index: 16, perplexity: inf
index: 17, perplexity: inf
index: 18, perplexity: inf
index: 25, perplexity: inf
index: 26, perplexity: inf
index: 27, perplexity: inf
index: 28, perplexity: inf
index: 29, perplexity: inf
index: 30, perplexity: inf
index: 44, perplexity: inf
index: 45, perplexity: inf
index: 46, perplexity: inf


In [7]:
x = list(pad_both_ends(test_tokenized_data[6], n=n))
x

['<s>',
 '<s>',
 'read',
 'world',
 "relief's",
 'response',
 'to',
 'the',
 'latest',
 'scotus',
 'opinion',
 'of',
 'refugee',
 'resettlement.https',
 ':/',
 '/',
 't.co/hiwv8hezm1',
 '</s>',
 '</s>']

In [8]:
for t in trigrams(x):
    print(t[2], t[:2])
    print(kn.score(t[2], t[:2]), "\n")

read ('<s>', '<s>')
0.0008325276637006617 

world ('<s>', 'read')
2.5845596623317364e-06 

relief's ('read', 'world')
0.01652240871350772 

response ('world', "relief's")
1.3903148528405205e-06 

to ("relief's", 'response')
0.4223072198591962 

the ('response', 'to')
0.003605245049780589 

latest ('to', 'the')
0.005289773508389742 

scotus ('the', 'latest')
3.948823250671301e-07 

opinion ('latest', 'scotus')
0.12857998435513837 

of ('scotus', 'opinion')
8.609257357973991e-05 

refugee ('opinion', 'of')
0.0021335010786854816 

resettlement.https ('of', 'refugee')
0.0 

:/ ('refugee', 'resettlement.https')
0.0006630732375085558 

/ ('resettlement.https', ':/')
0.9661373120487513 

t.co/hiwv8hezm1 (':/', '/')
0.0 

</s> ('/', 't.co/hiwv8hezm1')
0.02277977412731006 

</s> ('t.co/hiwv8hezm1', '</s>')
0.999954099566657 



In [9]:
kn.score("yousuck", ['#']) #1744

0.0

In [10]:
kn.score("yousuck", ['.', '#'])

0.0

In [11]:
kn.score("yousuck")

0.0

In [ ]:
x = Counter(train_list_hate).most_common()[::-1]
print(f"HATE\n\nvocabulary_size: {len(x)}\ncorpus size: {len(train_list_hate)}\ntoken/types ratio: {len(train_list_hate)/len(x)}")
y = Counter(train_list_sentiment).most_common()[::-1]
print(f"\nSENTIMENT\n\nvocabulary_size: {len(y)}\ncorpus size: {len(train_list_sentiment)}\ntoken/types ratio: {len(train_list_sentiment)/len(y)}")